In [1]:
import pandas as pd
import spacy,re,json
from flask import Flask, render_template, request
import pandas as pd
from pprint import pprint
from IPython.core.display import display, HTML

import warnings
warnings.filterwarnings('ignore')

# import NER model

In [2]:
ner_model = spacy.load('ner_model/')

# import Skill Hierarchical  relation

In [3]:
SKILL_Hierarchical_relation_DF = pd.read_excel(r'Hierarchical_relation_DF.xls')
SKILL_Hierarchical_relation_DF = SKILL_Hierarchical_relation_DF.drop_duplicates()
SKILL_Hierarchical_relation_DF

,node1,node2
0,data,Extract
1,Tableau,data visualization
2,Qlik,data visualization
3,algorithms,Deep learning
4,analytics,support
...,...,...
1174,machine learning,data
1187,statistics,data analytics
1189,design,A/B testing
1222,CSS3,CSS


# import Skill Correlation relation

In [4]:
# SKILL_Correlation_relation_DF = pd.read_excel(r'Correlation_relation_Link.xls')
# SKILL_Correlation_relation_DF = SKILL_Correlation_relation_DF.drop_duplicates()
# SKILL_Correlation_relation_DF

In [5]:
def get_HTML(my_doc, ent_list):
    total_html = ''
    for tem in my_doc:
        if  r'/r' in tem.text or  r'/n' in tem.text or  '\\r' in tem.text or '\\n' in tem.text:
            total_html += '<br/>'

        if tem not in ent_list:
            total_html += tem.text + ' '
        else:
            total_html += "<span style='background:yellow'>"+ tem.text +' ' + "</span>"

    display(HTML( '<html>'+ total_html + '</html>' ))
    return '<html>'+ total_html + '</html>'

In [6]:
def get_HTML_JD(my_doc, ent_list, cv_ent_list):
    total_html = ''
    for tem in my_doc:
        if  r'/r' in tem.text or  r'/n' in tem.text or  '\\r' in tem.text or '\\n' in tem.text:
            total_html += '<br/>'

        if tem not in ent_list:
            total_html += tem.text + ' '
        elif tem not in cv_ent_list:
            total_html += "<span style='background:red'>"+ tem.text +' ' + "</span>"
        else:
            total_html += "<span style='background:yellow'>"+ tem.text +' ' + "</span>"

    display(HTML( '<html>'+ total_html + '</html>' ))
    return '<html>'+ total_html + '</html>'

# Skill relation Dict

In [7]:
SKILL_relation_Dict = {}

for ind, row in SKILL_Hierarchical_relation_DF.iterrows():
    SKILL_relation_Dict[row['node1']] = row['node2']
    
    
# for ind, row in SKILL_Correlation_relation_DF.iterrows():
#     SKILL_relation_Dict[row['node1']] = row['node2']
    
SKILL_relation_Dict

{'data': 'Data consumption',
 'Tableau': 'data',
 'Qlik': 'Tableau',
 'algorithms': 'Data exploration',
 'analytics': 'architecture',
 'Software': 'Dashboards',
 'statistical modeling': 'data',
 'mechanics': 'Research',
 'relational databases': 'data',
 'SQL Server': 'MS Office',
 'Outlook': 'Microsoft Office',
 'Excel': 'MS Office',
 'Word': 'MS Office',
 'PowerPoint': 'MS Office',
 'Python': 'Data consumption',
 'Pandas': 'analytics',
 'MongoDB': 'Neo4j',
 'SQL': 'data analysis',
 'statistics': 'data analytics',
 'predictive modeling': 'data',
 'machine learning': 'data',
 'optimization': 'dashboards',
 'hypothesis testing': 'hypothesis testing',
 'R': 'Data consumption',
 'Matlab': 'statistical software',
 'Spotfire': 'data visualization',
 'TensorFlow': 'models',
 'Keras': 'Deep learning',
 'architecture': 'Minimum',
 'analysis': 'advanced analytics',
 'Python/R': 'software',
 'testing': 'Business Analytics',
 'Spark': 'Big Data',
 'H2O': 'learning',
 'MySQL': 'SQL Server',
 'neura

In [ ]:
app = Flask(__name__)

@app.route('/get_gap/', methods=['POST'])
# 返回最接近的向量
def get_simmilar():
    print( 'get_gap:' )
    
    print( 'JD:' )
    JD = eval(request.get_data())['JD']
    CV = eval(request.get_data())['CV']
    
    ## skill word in JD
    doc1 = ner_model(JD)
    JD_tem_list = []
    for chunk in doc1.ents:
        JD_tem_list.append( chunk.text )
    JD_tem_list = list(set(JD_tem_list))
    JD_ent_list = [t.root for t in doc1.ents]
    print( 'skill word in JD:', JD_tem_list)
    
    
    ## skill word in CV
    doc2 = ner_model(CV)
    CV_tem_list = []
    for chunk in doc2.ents:
        CV_tem_list.append( chunk.text )
    CV_tem_list = list(set(CV_tem_list))

    CV_tem_list_expand = []
    for tem in CV_tem_list:
        CV_tem_list_expand.append( str(tem) )
        if SKILL_relation_Dict.get( str(tem),None ) != None:
            CV_tem_list_expand.append( SKILL_relation_Dict.get( str(tem),None ) )

    CV_tem_list_expand = list(set(CV_tem_list_expand))

    CV_ent_list = [t.root for t in doc2.ents]
    CV_HTML = get_HTML( doc1, CV_ent_list)
    
    
    print( 'skill word in CV:', CV_tem_list_expand)
    CV_tem_list_expand = [w.lower() for w in CV_tem_list_expand]
    
    JD_HTML = get_HTML_JD( doc2, JD_ent_list, CV_tem_list_expand)
    
    from IPython.core.display import display, HTML
    display(HTML( JD_HTML  ))
    
    print( 'skill word in JD:', JD_tem_list)
    
    
    from IPython.core.display import display, HTML
    display(HTML( CV_HTML  ))
    
    print( 'skill word in CV:', CV_tem_list_expand)
    
    
    
    gap_tem_list = []
    for t1 in JD_tem_list:
        if t1.lower() not in CV_tem_list_expand:
            gap_tem_list.append( t1 )
    
    print('='*120)
    print( 'gap_list', gap_tem_list)
    response = { 
        'JD_HTML': JD_HTML,
        'CV_HTML': CV_HTML,
        'gap_list': gap_tem_list}
    
    return str(response)

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=5553)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5553/ (Press CTRL+C to quit)


get_gap:
JD:
skill word in JD: ['Verbal', 'Principles', 'Skills', "'s", 'etc', 'HTML5', 'Programming', 'Development', 'coding', 'CSS3', 'JQuery', 'SOAP', 'Multimedia', 'Interface', 'CSS', 'communication', 'cross', 'Web', 'compatibility', 'Services', 'HTML', 'Design', 'JavaScript', 'REST', 'User', 'Security', 'Object', 'Oriented', 'Job', 'Content', 'API', 'Communication', '-', 'Developer']


skill word in CV: ['Systems', 'MySQL5.0', 'Illustrator', 'design', 'HTML5', 'Basic', 'IDE', 'Windows', 'CSS3', 'Programming', 'data', 'Technologies', 'Notepad', 'JAVA', 'Draw', 'CSS', 'SKILLS', 'Data consumption', 'Text', 'Web', 'Bootstrap', 'Photoshop', 'C++', 'HTML', 'Sublime', 'bootstrap', 'JavaScript', 'detailed', 'A/B testing', 'C', 'Corel', 'Secondary', 'XP', 'Tools', 'Languages', 'usage', 'IT', 'jQuery', 'Operating']


skill word in JD: ['Verbal', 'Principles', 'Skills', "'s", 'etc', 'HTML5', 'Programming', 'Development', 'coding', 'CSS3', 'JQuery', 'SOAP', 'Multimedia', 'Interface', 'CSS', 'communication', 'cross', 'Web', 'compatibility', 'Services', 'HTML', 'Design', 'JavaScript', 'REST', 'User', 'Security', 'Object', 'Oriented', 'Job', 'Content', 'API', 'Communication', '-', 'Developer']


127.0.0.1 - - [28/Sep/2020 05:23:09] "POST /get_gap/ HTTP/1.1" 200 -


skill word in CV: ['systems', 'mysql5.0', 'illustrator', 'design', 'html5', 'basic', 'ide', 'windows', 'css3', 'programming', 'data', 'technologies', 'notepad', 'java', 'draw', 'css', 'skills', 'data consumption', 'text', 'web', 'bootstrap', 'photoshop', 'c++', 'html', 'sublime', 'bootstrap', 'javascript', 'detailed', 'a/b testing', 'c', 'corel', 'secondary', 'xp', 'tools', 'languages', 'usage', 'it', 'jquery', 'operating']
gap_list ['Verbal', 'Principles', "'s", 'etc', 'Development', 'coding', 'SOAP', 'Multimedia', 'Interface', 'communication', 'cross', 'compatibility', 'Services', 'REST', 'User', 'Security', 'Object', 'Oriented', 'Job', 'Content', 'API', 'Communication', '-', 'Developer']
